<img src="https://global.utsa.edu/tec-partnership/images/logos/logotipo-horizontal-azul-transparente.png"  width="600">

## **Actividad 3 | Aprendizaje supervisado y no supervisado**
### **Análisis de grandes volúmenes de datos (Gpo 10)**
#### Tecnológico de Monterrey
---
*   NOMBRE: Paulina Escalante Campbell
*   MATRÍCULA: A01191962
---
### **Objetivo**

Aplicar algoritmos de aprendizaje supervisado y no supervisado mediante PySpark para la resolución de problemas en análisis de datos, fomentando el desarrollo de habilidades prácticas en el manejo y procesamiento eficiente de grandes conjuntos de datos.

### **1. Introducción**

Introducción teórica. Se deberá de documentar de forma breve los conceptos de aprendizaje supervisado, no supervisado, así como los algoritmos más representativos que se identifican en la literatura de cada tipo de aprendizaje, además de identificar aquellos que están disponible a través de PySpark.
Proporciona una introducción clara y detallada, alineada completamente con el objetivo de la actividad. Incluye conceptos teóricos relevantes y bien explicados.

#### **Imports**


In [15]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

import findspark
findspark.init()
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, sum, when, split, col, lit, max, min, expr
from pyspark.sql.functions import to_date, var_samp, variance, var_pop, month, to_timestamp, dayofweek
from pyspark.sql.types import NumericType, IntegerType, FloatType
from pyspark.sql.functions import col, round, concat_ws, desc, when, concat
from pyspark.sql import functions as F

### **2. Selección de los datos**

Para esta actividad, se propone recolectar una muestra de dimensión contenida (para evitar que los tiempos de procesamiento sean altos) a partir de la base de datos que estás trabajando en tu proyecto. Para ello y tomando como base la actividad previa en la cual has implementado códigos que permiten obtener particiones de la base de datos global D que cumplen con los criterios de las variables de caracterización identificadas, se propone que recuperes un número limitado de instancias de cada partición (aplicando la técnica de muestreo que propusiste en el Módulo 3, Proyecto Base de datos de Big Data, paso 4), lo que te permitirá construir una muestra M a partir de la unión de las instancias que se recuperan de este proceso.

Selección de los datos: código, con documentación, donde se implementan las etapas para la construcción de muestra inicial M (punto 2)

Implementa y documenta de manera excelente el código para la selección de datos, justificando cada paso con claridad y mostrando una comprensión profunda del proceso.

In [18]:
spark = SparkSession.builder \
    .appName("Maestria_evidencia1") \
    .config("spark.driver.memory", "64g") \
    .config("spark.executor.memory", "32g") \
    .config("spark.sql.shuffle.partitions", "32") \
    .config("spark.default.parallelism", "16") \
    .config("spark.driver.maxResultSize", "16g") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Mejores tablas
#spark, comentando el comando del environment para reducir el ruido del notebook

In [10]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

####
#### La base de datos ha sido limpiada y modificada a este punto
#### 
####
file_path = "/Users/pauescalante/Documents/Maestria/Trimestre 7/BigData/big-data-act/DataModified/expanded_database_ecommerce"
df = spark.read.csv(file_path, header=True, inferSchema=True)

df.head()

/opt/anaconda3/envs/env-pyspark/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
25/05/21 21:46:03 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
                                                                                

Row(event_time=datetime.date(2019, 11, 17), event_type='view', product_id=5300440, category_id=2053013563173241677, brand='vitek', price=17.76, user_id=513341639, user_session='d9544029-2739-4d16-9cac-79650460d9f0', event_time_ts=datetime.datetime(2019, 11, 17, 5, 35, 32), parent_category='None', subcategory='None', price_bucket='low', day_of_week=1, is_weekend=True)

In [11]:
df = df.selectExpr(
    'cast(event_time as timestamp) event_time',
    'cast(event_type as string) event_type',
    'cast(product_id as int) product_id',
    'cast(category_id as long) category_id',
    'cast(brand as string) brand',
    'cast(price as float) price',
    'cast(user_id as int) user_id',
    'cast(user_session as string) user_session',
    'cast(parent_category as string) parent_category',
    'cast(subcategory as string) subcategory',
    'cast(price_bucket as string) price_bucket',
    'cast(event_time_ts as timestamp) event_time_ts',
    'cast(day_of_week as int) day_of_week',
    'cast(is_weekend as boolean) is_weekend',
)

df.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: float (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)
 |-- parent_category: string (nullable = true)
 |-- subcategory: string (nullable = true)
 |-- price_bucket: string (nullable = true)
 |-- event_time_ts: timestamp (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- is_weekend: boolean (nullable = true)



### **3. Preparación de los datos**

En esta etapa, se deberán de aplicar estrategias de corrección sobre los datos que integran a la muestra M que se ha preparado en el paso previo, de tal forma que de deje un conjunto M listo para ser procesado por los algoritmos de aprendizaje a aplicar. Para ello se deben de considerar pasos como: corrección de registros / columnas con valores nulos, identificación de valores atípicos, transformación de los tipos de datos, etc. Con lo anterior, se tendrá una muestra M pre-procesada.

Etapa donde se pre-procesa la muestra M, para corregir formatos e inconsistencias de cualquier índole que tenga la muestra original. Se deberá de documentar los pasos que se implementen para resolver esta etapa.

Pre-procesa la muestra M corrigiendo todos los formatos e inconsistencias con documentación detallada y justificada de cada paso.

In [12]:
df.head()

Row(event_time=datetime.datetime(2019, 11, 17, 0, 0), event_type='view', product_id=5300440, category_id=2053013563173241677, brand='vitek', price=17.760000228881836, user_id=513341639, user_session='d9544029-2739-4d16-9cac-79650460d9f0', parent_category='None', subcategory='None', price_bucket='low', event_time_ts=datetime.datetime(2019, 11, 17, 5, 35, 32), day_of_week=1, is_weekend=True)

### **4. Preparación del conjunto de entrenamiento y prueba**

Para esta etapa, la muestra M será divida en un conjunto de entrenamiento y prueba. Para ello, deberás proponer una técnica de muestreo que te permita construir el conjunto de entrenamiento y prueba minimizando el riesgo de inyección de sesgos. Ten en cuenta que, para este punto, deberás de tener en claro el porcentaje de división a utilizar, el cual se deberá de justificar.

Preparación del conjunto de entrenamiento y prueba: código en la cual se construye el conjunto de entrenamiento y prueba para la experimentación, debidamente documentado el código.

Construye y documenta de manera excelente el conjunto de entrenamiento y prueba, con una explicación clara y justificada de cada paso.

In [13]:
df.head()

Row(event_time=datetime.datetime(2019, 11, 17, 0, 0), event_type='view', product_id=5300440, category_id=2053013563173241677, brand='vitek', price=17.760000228881836, user_id=513341639, user_session='d9544029-2739-4d16-9cac-79650460d9f0', parent_category='None', subcategory='None', price_bucket='low', event_time_ts=datetime.datetime(2019, 11, 17, 5, 35, 32), day_of_week=1, is_weekend=True)

### **5. Construcción de modelos de aprendizaje supervisado y no supervisado**

Para este punto realizarás dos experimentos separados, dónde se aplicará un algoritmo de aprendizaje supervisado y uno de aprendizaje no supervisado sobre la muestra M. Para el caso de aprendizaje supervisado, se deberá de identificar cuál es la variable objetivo (columna) de aprendizaje, mientras que, para el caso de aprendizaje no supervisado, se debe de seleccionar todas las columnas que se desean considerar como características bajo las cuales se realizará el proceso de agrupamiento. Usando las implementaciones correspondientes de PySpark, se deberá de ejecutar el aprendizaje correspondiente a partir de la invocación de las funciones respectivas. Para este ejercicio, se deberá seleccionar un criterio básico para medir la calidad del resultado obtenido, dependiendo de cada tipo de aprendizaje implementado. La elección quedará a juicio de cada estudiante.

Construcción de modelos de aprendizaje supervisado y no supervisado: se recomienda dividir en dos subsecciones, una donde se muestre como se entrena un algoritmo de aprendizaje supervisado, y otra para el entrenamiento de un algoritmo no supervisado. Se debe de documentar el código implementado con una breve discusión de resultados.

Entrena y documenta de manera excelente ambos tipos de algoritmos, con una discusión detallada y clara de los resultados obtenidos.

In [14]:
df.head()

Row(event_time=datetime.datetime(2019, 11, 17, 0, 0), event_type='view', product_id=5300440, category_id=2053013563173241677, brand='vitek', price=17.760000228881836, user_id=513341639, user_session='d9544029-2739-4d16-9cac-79650460d9f0', parent_category='None', subcategory='None', price_bucket='low', event_time_ts=datetime.datetime(2019, 11, 17, 5, 35, 32), day_of_week=1, is_weekend=True)